In [59]:
import csv
import json
import requests
#import elasticsearch
from elasticsearch import Elasticsearch as ES
from elasticsearch import helpers

ESURL='http://localhost:9200'
ESIDX='hai-idx'
CURL="curl --silent -H Content-Type:application/json"

# es_client = elasticsearch.Elasticsearch(ES)
es_client = ES(ESURL)
# print(es_client)
print(es_client.indices.get_alias().keys())

def postHaiIdx(row) :
    #print(row)
    es_client.create(index=ESIDX,doc_type="doc", id=row["id"],
                    body={
                        'indx': row["id"],
                        'intent' : row["intent"],
                        'utterance' : row["utterance"]
                    })
    
    #print("don postHaiIdx")
    
def makeHaiIdx() :
    #es_client.indices.delete(index="hai-idx")
    
    csv_training = open('./data/hai_training.csv', 'r', encoding='utf-8-sig')
    print(csv_training)
    reader = csv.DictReader(csv_training)
    totalrows = 0
    for row in reader:
        postHaiIdx(row)
        #print(row["utterance"])
        totalrows += 1
        
    print(totalrows)

    csv_training.close()


#def compareWithTraining(row, TCNT, FCNT) :
def compareWithTraining(row) :
    srch_docs = es_client.search(index = ESIDX, doc_type = 'doc',
                       body = {
                           'query': {
                               'match': {
                                   #'utterance': '컨설팅팀 알려'
                                   'utterance': {
                                       #'query': row["utterance"], 'operator': 'and'
                                       'query': row["utterance"]
                                   }
                               }
                           }
                       })
    #print("len : {}".format(len(srch_docs["hits"]["hits"])))
    
    resultBolean = False;
        
    if len(srch_docs["hits"]["hits"]) > 0 :
        #resultBolean = False;
        '''
        print("result max:{}, score:{}, intent:{}".format(
                 srch_docs["hits"]["max_score"]
                ,srch_docs["hits"]["hits"][0]["_score"]
                ,srch_docs["hits"]["hits"][0]["_source"]["intent"]))
        '''
        if row["intent"] == srch_docs["hits"]["hits"][0]["_source"]["intent"] :
            #print("True")
            resultBolean = True;
        else :
            #print("False")
            resultBolean = False;
        '''
        print("result [{}] max:{}, score:{}, intent:[{}][{}]".format(
                 resultBolean
                ,srch_docs["hits"]["max_score"]
                ,srch_docs["hits"]["hits"][0]["_score"]
                ,srch_docs["hits"]["hits"][0]["_source"]["intent"]
                ,row["intent"]))
        '''    
        maxs   = srch_docs["hits"]["max_score"]
        score  = srch_docs["hits"]["hits"][0]["_score"]
        intent = srch_docs["hits"]["hits"][0]["_source"]["intent"]
        uttera = srch_docs["hits"]["hits"][0]["_source"]["utterance"]
        
        print("[{}], [{}][{}], [{}][{}]".format(resultBolean, row["intent"], intent, row["utterance"], uttera))
    else :
        print("[None] [{}],[{}]".format(row["intent"],row["utterance"]))
        
    return resultBolean
    
def checkHaiIdxWithTest() :
    #es_client.indices.delete(index="hai-idx")
    
    #tCnt = 0
    #fCnt = 0
    
    csv_test = open('./data/hai_test.csv', 'r', encoding='utf-8-sig')
    #print(csv_test)
    reader = csv.DictReader(csv_test)
    totalrows = 0
    tCnt = 0
    fCnt = 0
    
    for row in reader:
        RCNT = compareWithTraining(row)
        if RCNT :
            tCnt += 1
        else :
            fCnt += 1
            
        #compareWithTraining(row)
        #print(row["utterance"])
        totalrows += 1
        
    print("hai_test : {}, {}, {}".format(totalrows, tCnt, fCnt))

    
    csv_test.close()
    
#makeHaiIdx()
checkHaiIdxWithTest()


# print(es_client.indices.get_alias().keys())

#Get a document
#https://docs.objectrocket.com/elastic_python_examples.html
#http://www.pythonexample.com/search?q=elasticsearch
# http://jason-heo.github.io/elasticsearch/2016/07/16/elasticsearch-with-python.html

#get_doc = es_client.get(index=ESIDX, doc_type='doc', id='2322')
#print(get_doc)
#print("\n")

# Search (DSL)
srch_docs = es_client.search(index = ESIDX, doc_type = 'doc',
                       body = {
                           'query': {
                               'match': {
                                   #'utterance': '컨설팅팀 알려'
                                   'utterance': {
                                       'query': '통제예산 알려', 'operator': 'and'
                                   }
                               }
                           }
                       })
print("len : {}".format(len(srch_docs["hits"]["hits"])))
#print("result count : {}".format(srch_docs["hits"]["total"]))

#if srch_docs is not None :
if len(srch_docs["hits"]["hits"]) > 0 :
    print("result max:{}, score:{}, intent:{}".format(
                 srch_docs["hits"]["max_score"]
                ,srch_docs["hits"]["hits"][0]["_score"]
                ,srch_docs["hits"]["hits"][0]["_source"]["intent"]))
        
#print(srch_docs)
#print(json.dumps(srch_docs, indent = 4, ensure_ascii=False))
#print(json.dumps(srch_docs["hits"]["hits"][0], indent = 4, ensure_ascii=False))
#print(json.dumps(srch_docs))

#Delete a document
#es_client.delete(index=ESIDX, doc_type='doc', id=3000)



dict_keys(['.monitoring-es-6-2018.01.17', 'seunjeon-idx', 'hai-idx', '.monitoring-es-6-2018.01.19', '.monitoring-es-6-2018.01.18'])
[None] [None],[;;;?]
[True], [None][None], [오늘 몇 일이야?][오늘 몇일 이지?]
[True], [None][None], [멍청한 새x....][멍청하네]
[True], [None][None], [다모르는거면 언제답해줄거니?][다 모르니]
[None] [None],[우울]
[None] [None],[가나]
[None] [None],[야호]
[False], [None][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [서비스데스크][서비스 데스크]
[True], [None][None], [ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ똘똘아 좀 닥치고있어봐 너 말이 너무 길어][ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ]
[True], [None][None], [맞는다?][^^ 반가와요~ 근데 진짜 AI가 맞나요?]
[False], [None][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [부서 에싼][예싼]
[False], [None][ECS_SNC_INFORMATION], [하세요...][뭐뭐하니]
[True], [None][None], [그래.. 넌 몇살이야?][몇살이야?]
[True], [None][None], [점심에 어디가?][점심]
[True], [None][None], [양념치킨 vs 후라이드치킨][양념치킨 vs 후라이드치킨]
[True], [None][None], [퇴근하고 싶어요][퇴근하고 싶어]
[True], [None][None], [졸려요][졸리다]
[False], [None][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [회계팀][회계팀]
[True], [None][None], [이야!!!][그럼 이만]
[True],

[True], [None][None], [테스트][테스트]
[True], [None][None], [답변][답변]
[False], [None][ECS_SNC_INFORMATION], [똘똘이 이름바꿔줄까?][이름이 한똘똘이야?]
[True], [None][None], [내가 그거 물어봤니?][아니.. 그거 말고..]
[False], [None][ECS_SNC_INFORMATION], [ai][너 AI야?]
[True], [None][None], [HANWHA AI][hanwha-ai]
[False], [None][ECS_SNC_GREETING], [ㅇ][ㅇ]
[True], [None][None], [바이][바이~~]
[False], [None][ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS], [품의][출장품의]
[True], [None][None], [지금][퇴지금]
[False], [None][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [서비스데스크 등록방법][서비스 데스크]
[True], [ECS_SNC_WEATHER_FORECAST][ECS_SNC_WEATHER_FORECAST], [오늘 날씨가 어때?][오늘 날씨가 어때요?]
[True], [ECS_SNC_WEATHER_FORECAST][ECS_SNC_WEATHER_FORECAST], [경남창원 날씨 알려줘][오늘 경남 창원 날씨는?]
[True], [ECS_SNC_WEATHER_FORECAST][ECS_SNC_WEATHER_FORECAST], [여의도 날씨][여의도 날씨]
[True], [ECS_SNC_WEATHER_FORECAST][ECS_SNC_WEATHER_FORECAST], [내일 날씨 알려줘][내일날씨?]
[True], [ECS_SNC_WEATHER_FORECAST][ECS_SNC_WEATHER_FORECAST], [을지로 날씨 어때?][을지로 날씨]
[True], [ECS_SNC_WEATHER_FORECAST][ECS_SNC_WE

[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [대표이사 알려줘][대표이사 알려주세요]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], ['건강검진'][건강검진]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [스마트폰 담당자는 누구야?][스마트 팩토리 TF 담당자]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [그룹웨어 담당자][그룹웨어 담당자]
[False], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [최범진 과장 경비 처리 현황은?][경비 처리 현황]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [문서보안 담당자가 누구][문서보안 담당자가 누구에요?]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [법무담당자][법무 담당자]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_T

[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [한건숙 과장님은 몰라?][혹시 한건숙 과장님 아세요?]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [PI팀원][PI팀]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [김정원][김정원]
[None] [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE],[이수미]
[False], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_APPROVAL_FORM_LINK], [출장비 담당][출장비]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [PC보안 담당자][보안 담당자]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [신윤정][신윤정]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [문서보안 담당자][문서보안 담당자]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE][ECS_

[False], [ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS][None], [우리회사에서 여의도까지 가는 가장 빠른 방법][우리회사 주소는?]
[False], [ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS][None], [비쥬얼 스튜디오 라이센스 있니?][있는데?]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS][ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS], [미국 지도가 궁금합니다][미국 지도]
[False], [ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS][None], [평가 제도][평가]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS][ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS], [국내출장비 규정?][출장비 규정이 궁금해]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS][ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS], [네트워크 설정][무선네트워크]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS][ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS], [경조휴가 대상은?][경조휴가 휴가]
[True], [ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS][ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS], [인사 제도][인사제도]
[False], [ECS_SNC_SS1_CIRCLE_INQUIRY_SUPPORTING_BUSINESS][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_

[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [팀 잔여예산][팀 잔여예산]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [야근자식대][야근자식대]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [나 휴가 얼마나 남았니?][내 연자 얼마나 남았지?]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산은][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [잔여예산][잔여 예산]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGE

[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [비통제예산][통제예산]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산][예산은]
[False], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_SS1_CIRCLE_INQUIRY_THE_PERSON_IN_CHARGE], [김준우 대리 PM비용 알려줘][한화케미칼 개인정보DB암호화및접근제어 PM비용 알려주세요]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [부서 예산 얼마 남았어][예산 얼마 남았니]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산은][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [예산][예산은]
[True], [ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET][ECS_SNC_PI_ERP_INQUIRY_TEAM_BUDGET], [통제경비][통제성경비]


[True], [ECS_SNC_INFORMATION][ECS_SNC_INFORMATION], [넌 아는게 뭐니?][아는게 뭐야]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [안녕][안녕하세요 ?]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [안녕하세요][안녕하세요 ?]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [똘똘아 안녕][똘똘안녕]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [안녕][안녕하세요 ?]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [안녕하세요?][안녕하세요 ?]
[False], [ECS_SNC_GREETING][None], [미안][미안하다..]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [안녕][안녕하세요 ?]
[False], [ECS_SNC_GREETING][None], [HR][U-HR]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [안녕][안녕하세요 ?]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [안녕][안녕하세요 ?]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [똘똘씨][한똘똘씨~]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [안녕하세요][안녕하세요 ?]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [똘똘아][똘똘아 ~~~]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [안녕][안녕하세요 ?]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING], [안녕][안녕하세요 ?]
[True], [ECS_SNC_GREETING][ECS_SNC_GREETING